In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd


# ACTIVITY 6

In [ ]:

Data_Ana = pd.read_csv('Data/Data_Marketing_Customer_Analysis_Round3.csv')
Data_Ana_Clean = Data_Ana.drop(['Unnamed: 0'], axis = 1)
Data_Ana_Clean

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Data_Marketing_Customer_Analysis_Round3.csv'

## 1. One Hot/Label Encoding of the categorical variables in the categoricals data frame that you created in Activity 4. ( Hint: check Dropfirst option).

In [ ]:
categorical_data = Data_Ana_Clean.select_dtypes('object')
categorical_data

In [ ]:
dummy_data = pd.get_dummies(categorical_data)
dummy_data

## 2. for the first iteration, we will focus on the numericals data (numericals data frame) as our data source.

In [ ]:
numerical_data = Data_Ana_Clean._get_numeric_data()
numerical_data

In [ ]:
# customer_lifetime_value = numerical_data['customer_lifetime_value']
# income = numerical_data['income']
# monthly_premium_auto = numerical_data['monthly_premium_auto']
# months_since_last_claim = numerical_data['months_since_last_claim']
# months_since_policy_inception = numerical_data['months_since_policy_inception']## 3. remove the outliers from the numerical columns.


# from scipy.stats import scoreatpercentile as pct
# from scipy.stats import iqr

# def remove_outliers_costum():
#     pct_75 = pct(customer_lifetime_value, 75)  # Calculate percentile 75 using scipy function scoreatpercentile
#    pct_25 = pct(customer_lifetime_value, 25)  # Calculate percentile 25 using scipy function scoreatpercentile
#    upper_bound = pct_75 + 1.5*iqr(customer_lifetime_value)  # iqr - > Scipy function to calculate the Interquartile Range
#    lower_bound = pct_25 - 1.5*iqr(customer_lifetime_value)
#    customer_lifetime_value2 = customer_lifetime_value[(customer_lifetime_value <= upper_bound) & (customer_lifetime_value >= lower_bound)]  # Filter out the outliers
#    return customer_lifetime_value2


# remove_outliers_costum()

# NOTE: this works for only singular columns


In [ ]:
# this is for removing the ouliers from all the columns.

Q1 = numerical_data.quantile(0.25)
Q3 = numerical_data.quantile(0.75)
IQR = Q3 - Q1

numerical_data = numerical_data[~((numerical_data < (Q1 - 1.5 * IQR)) |(numerical_data > (Q3 + 1.5 * IQR))).any(axis=1)]

numerical_data

In [ ]:
# or simimlarly we can take only the non-outliers values:

# numerical_data = numerical_data[((numerical_data > (Q1 - 1.5 * IQR)) |(numerical_data < (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
# checkup of some values

print(numerical_data['number_of_open_complaints'].unique())

## 4. check if there are highly correlated features and drop them if there are any.


In [ ]:

corr = numerical_data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(9, 7))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)
    

# RESULT: there are no correlations.

## 5. X-y split (y is the target variable which is the total claim amount)

In [ ]:


# first I create a data frame with 
#. X= all numerical data without the target ('total_claim_amount') and 
#. y= total_claim_amount.


# axis=1  is the X axes and axis=0 is the Y axes.



In [ ]:
X = numerical_data.drop('total_claim_amount', axis=1)
y = numerical_data.total_claim_amount

X

In [ ]:
y

## Train / Test Split - ACTIVITY 7

In [ ]:

# Now we split the data in TEST and TRAIN sets (in agreement with Hila and Fra  ;)   ).

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,random_state=123)

### test size is the percentage of data taken as test_set (30% test and 70% training, in this case).
### you can also write 'train_size =.70' .
### random_state = any number you put is the same. this function keep always the same random choice of data taken as sets.
###                otherwise it takes always different values when you run it.

# check rows and columns
X_train.shape



In [ ]:
X_test.shape

## Finding the parameters (mean, variance, from the training set) - ACTIVITY 6

## Standardize the data (after the data split) - ACTIVITY 7
Standardize the data (after the data split). (done above).
Standardize the Data is necessary to bring all the Data to the same levels (to reduce the variables) to be compared each other.
(indicateed below as x_test_scaled or x_train_scaled).

In [ ]:


std_scaler = StandardScaler().fit(X_train)   # it calculates the z score.

X_train_scaled = std_scaler.transform(X_train)   # transforming the data: normalization of all data. 


X_test_scaled = std_scaler.transform(X_test)

In [ ]:

# check rows and columns
X_train.shape

In [ ]:
print(X_train_scaled)
print("--------")
print(X_test_scaled)

# ACTIVITY 7

In [ ]:
X.columns

In [18]:
# Modeling using Statsmodels

X_train_const_scaled = sm.add_constant(X_train_scaled)     # adding a constant

model = sm.OLS(y_train, X_train_const_scaled).fit()
predictions_train = model.predict(X_train_const_scaled) 

X_test_const_scaled = sm.add_constant(X_test_scaled)       # adding a constant
predictions_test = model.predict(X_test_const_scaled) 
print_model = model.summary()
print(print_model)

NameError: name 'X_train_scaled' is not defined

In [19]:
predictions_test

NameError: name 'predictions_test' is not defined

### R-SQUARED VALUE

In [ ]:
# display adjusted R-squared
# difference between the model and the real values (the training data set).
# to check the validity of the model.

print(model.rsquared_adj)

## Apply linear regression.

In [ ]:
# (This is a Supervised Training. So you need the X_train_scaled (all standardized training data) and the values used as target, which are the y_train).


model = LinearRegression()            # model
model.fit(X_train_scaled, y_train)    # model train



In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
# then you get the following model that can predict new data.
# NOTE: the values in the equation below come from the model.coef_ and the model.intercept_  

# target_donation = 1.34* avg_fam_income  -0.41 * pobc2   +5.45 * avggift  -0.48 * med_home_val_transformed  + 15.66

In [ ]:
# the following 'predict function' is running the equation that is above (the model for predict new data).

y_pred=model.predict(X_test_scaled)          # model prediction

y_pred_train=model.predict(X_train_scaled)

In [ ]:
X

## Model Interpretation.

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
residual = y_test - y_pred
residual

In [ ]:
result=pd.DataFrame({"y_test":y_test,"y_pred":y_pred})

In [ ]:
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the all the predictions are ok?
# A stright line

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1].hist(residual)        ## residual = (y_test - y_pred)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred,residual,"o")
ax[2].set_xlabel("predicted")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')

# Explanation of the graphs are in the file: 'Notebook_Code_Along_Intro_To_Pandas_Healthcare For_All_Modeling'.

In [ ]:
# note: when a model is really good:
####   - first graph: model works in prediction when it result a linear regression between the predict and the test values.
####   - second graph: histogram; perfect if normal distribution of the errors (y_test-y_pred).
####   - last graph: plotting the 'residuals' with the 'predict', more the values are around the line more it means that the residuals (errors) are zero (zero difference between prediction and real values), and more the model is good.



## Seaborn alternative to check the linear regression model

In [ ]:
sns.regplot(x='y_pred',y='y_test', data=result, scatter_kws={"color": "salmon"}, line_kws={"color": "purple"})

## MSE: Mean Squared Error
## MAE: Mean Absolute Error
## RMSE: Root Mean Square Error 

In [ ]:
# Predictions on the test set

print(mse(y_test,y_pred))
print(mae(y_test,y_pred))
print(np.sqrt(mse(y_test,y_pred)))   ## in Seaborn doesn't exist RMSE so we calculate it with numpy.

In [ ]:
# Prediction on the train set

mse(y_train,y_pred_train)

In [ ]:
mae(y_train,y_pred_train)

In [ ]:
## if the errors in train and test sets are similar the models are ok.

# R2: Sample R-Squared

R2: Sample R-Squared
- Provides a way to compare the performance of several models
- Provides a measurement of the “explanatory” power of a model (shows how much variance of y can be explained by the independent features) 
- Compares the variance of your model’s errors against the “mean model” (model for which all the predictions are the mean of the dependent variable)
- Increases with every independent feature you add
- more the R2 value is close to 1 and more the model is perfect.


note: see presentation week4 day2 (Linear Regression).

R^2 = (SST - SSE)/SST


R^2 = 1-SSE/SST

more the R2 value is close to 1 and more the model is perfect.

SSE = sum of the square of the errors
SST = total sum of squares explained by the model

In [ ]:
R2=r2_score(y_test,y_pred)
R2

In [ ]:
R2_test=model.score(X_test_scaled,y_test)
R2_train=model.score(X_train_scaled,y_train)

In [ ]:
R2_test

In [ ]:
R2_train

### adjusted R2:

Provides a measurement of the “explanatory” power of a model taking into account the number of independent features used by the model.
Only increases if the inclusion of a new feature improves the model’s performance.
- it is used for Sklearn since it doesn't have R2.


In [ ]:
adjusted_R2_test = 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
adjusted_R2_test


# Feature Importances
#### it helps to check which variable is more affecting the model. It means, when little changes of the variable bring bigger changes to the result of the model equation.

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)

In [ ]:
features_importances

In [ ]:
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()